In [1]:
import os
from src.reddit import MyRedditAPI
from src.processing import EntityRecognition, clean_text

##### Step 1: Create an instance of the MyRedditAPI Class
- Enter your own client_id, secret_key, username, and password
    - To create the Client ID and Secret Key: https://www.reddit.com/prefs/apps
- For source code -> src.reddit.my_reddit_api

In [2]:
session = MyRedditAPI(
    client_id=os.getenv("CLIENT_ID"),  # API Client
    secret_key=os.getenv("SECRET"),    # API Secret
    username=os.getenv("USERNAME"),    # Reddit Username
    password=os.getenv("PASSWORD"),    # Reddit Account Password
    user_agent='MyApiTest'             # Can be anything
)

##### Step 2: Send a request for data
- Note that it is only possible to retrieve the number of posts that are available in reddit's "main search". For example, if you search anything on reddit, the maximum number of posts that will be returned is ~250. So, even if the limit is set higher than that, it will not return more than what is available to be searched in the application.
- Defult behavior is to search all subreddits, here are some others to try:
    - subreddit="ukraine"
    - subreddit="RussiaUkraineWar2022"
    - subreddit="UkraineConflict"

In [3]:
wagner_df = session.search_posts(
    query='Wagner Group',
    sort='new',
    subreddit='all',
    limit=300
)

##### Step 3: Text Cleaning

In [4]:
wagner_df = clean_text(wagner_df, columns=['title', 'selftext'])

##### Step 4: Named Entity Recognition to extract locations, organizations, and people from the title (or selftext if desired)
- Note: may need to download the en_core_web_sm spacy model: python -m spacy download en_core_web_sm

In [5]:
wagner_ner = EntityRecognition(wagner_df, 'title')
wagner_df['locations'] = wagner_ner.get_gpe()
wagner_df['organizations'] = wagner_ner.get_org()
wagner_df['people'] = wagner_ner.get_person()

In [8]:
wagner_df

,name,created_utc,author,subreddit,title,selftext,media_url,post_url,locations,organizations,people
0,t3_zkbxpf,2022-12-12 21:22:43,Few_Mulberry7175,NBA_Draft,My 1st 2023 mock draft (Lottery only),"With it quickly reaching Christmas, I thought ...",https://www.reddit.com/r/NBA_Draft/comments/zk...,https://www.reddit.com/r/NBA_Draft/comments/zk...,,,
1,t3_zk8cyt,2022-12-12 19:14:05,SesshamoNekodearuzo,what_couldve_been_if,Huge losses. Wagner Group headquarters blown b...,,https://www.youtube.com/watch?v=apxhEd6l-JE,https://www.reddit.com/r/what_couldve_been_if/...,,wagner group,
2,t3_zk6fxy,2022-12-12 18:06:02,AutoNewspaperAdmin,AutoNewspaper,[World] Ukraine Says It Hit Hotel Where Member...,,https://www.nytimes.com/2022/12/12/world/europ...,https://www.reddit.com/r/AutoNewspaper/comment...,"ukraine, russia",wagner group gathered ny times,
3,t3_zk64g0,2022-12-12 17:55:27,AutoNewsAdmin,NYTauto,[World] Ukraine Says It Hit Hotel Where Member...,,https://www.nytimes.com/2022/12/12/world/europ...,https://www.reddit.com/r/NYTauto/comments/zk64...,"ukraine, russia",wagner group,
4,t3_zk5mr5,2022-12-12 17:37:32,True-Cucumber6316,infantile,the real movement to want to kill myself,,https://i.redd.it/9is4yhvynj5a1.jpg,https://www.reddit.com/r/infantile/comments/zk...,,,
...,...,...,...,...,...,...,...,...,...,...,...
223,t3_zb8kxt,2022-12-03 05:24:04,safamannvideos,u_safamannvideos,Armed Forces of Ukraine hit the column of the ...,,https://www.febspot.com/129519,https://www.reddit.com/r/u_safamannvideos/comm...,,"armed forces of ukraine, the russian army wagn...","kherson oblast, zaporizhzhia"
224,t3_zb4yqb,2022-12-03 02:13:09,A33Bot,other_discussions,[Themes] Spain on high alert after series of l...,\#LetterBombThreat: People are alarmed by the ...,https://www.reddit.com/r/other_discussions/com...,https://www.reddit.com/r/other_discussions/com...,"ukraine, spain",guardian,
225,t3_zb3njd,2022-12-03 01:09:01,-en-,newsbotbot,@AP: The Biden administration on Friday placed...,,https://twitter.com/AP/status/1598836961569120256,https://www.reddit.com/r/newsbotbot/comments/z...,,the wagner group,biden
226,t3_zaxavr,2022-12-02 20:54:41,AutoNewspaperAdmin,AutoNewspaper,[Politics] State Department adds Russian merce...,,https://www.cnn.com/2022/12/02/politics/us-rus...,https://www.reddit.com/r/AutoNewspaper/comment...,,"cnn, wagner group",
